In [ ]:
import tensorflow as tf
from keras import models, layers
import matplotlib.pyplot as plt
import numpy as np

Makra pro vyuziti v modelu

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 64
CHANNELS = 3    #RGB 
EPOCHS = 10

Nacte dataset


In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "dataset",
    shuffle=True,   #mix the input pictures
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Nacteni jmen vsech class a ukazka 12ti obrazku z nacteneho datasetu s jejich popiskem

In [ ]:
class_names = dataset.class_names

def print_elements_from_dataset(dataset, class_names, FIG_SIZE=15):
    print(class_names)
    plt.figure(figsize=(FIG_SIZE,FIG_SIZE))
    for image_batch, label_batch in dataset.take(1):
        for i in  range(12) :
            plt.subplot(3,4,i+1)
            plt.imshow(image_batch[i].numpy().astype("uint8"))
            plt.title(class_names[label_batch[i]])
            plt.axis("off")
            
print_elements_from_dataset(dataset, class_names)

Rozdel dataset na partitions 
prevzato z internetu

In [ ]:
def get_partitions_of_dataset(dataset, train_split=0.8, val_split = 0.1, shuffle=True, shuffle_size=10000):
    
    dataset_size = len(dataset)
    #nevim jestli zase shuffle kdyz uz byl pouzit vyse
    if shuffle:
        dataset = dataset.shuffle(shuffle_size, seed=12)

    train_size = int(train_split*dataset_size)
    val_size = int(val_split*dataset_size)

    train_ds = dataset.take(train_size)

    val_ds = dataset.skip(train_size).take(val_size)

    test_ds = dataset.skip(train_size+val_size)

    return train_ds, val_ds, test_ds

train_ds , val_ds, test_ds = get_partitions_of_dataset(dataset)
len(train_ds)

Layers for the actuall model
efektivita modelu, cache - nactena data jsou v mezipameti, shuffle - michani podle dane velikosti, prefetch - asynchrnni nacitani do pametti

In [ ]:
#efektivita modelu, cache - nactena data jsou v mezipameti, shuffle - michani podle dane velikosti, prefetch - asynchrnni nacitani do pametti
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

Otazka jestli ma smysl delat u tohoto datasetu, resize a rescale 

Proc delat resizing kdyz uz bylo udelano pri nacitani datasetu


In [ ]:
resize_and_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

Nevyuziji z duvodu velikosti dat, zkusime pri dalsim treninkk, jak to ovlivni vysledek

0.3 - o kolik se natoci 


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomRotation(0.3),
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
])

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

n_classes = len(class_names)

model = models.Sequential([
    resize_and_rescale,
    #data_augmentation,
    layers.Conv2D(32, (3,3), activation="relu", input_shape = input_shape),   # Conv2D(number of filters (trial and error), )
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size= (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, kernel_size= (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense (64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])
model.build(input_shape=input_shape)

Ukazka upravenych obrazku, momentalne nevyuzite, protoze ji nevyuzivam v modelu vyse

In [ ]:

# Funkce pro zobrazení několika obrázků z datasetu
def show_augmented_images(dataset, num_images=16):

    # Vytvoření iteratoru pro získání dat z datasetu
    iterator = iter(dataset)
    
    # Načtení několika obrázků z datasetu
    images = next(iterator)[0][:num_images]

    # Vytvoření obrázků před a po augmentaci
    original_images = resize_and_rescale(images)
    augmented_images = data_augmentation(original_images)

    # Zobrazení obrázků
    for i in range(num_images//2):
        
        plt.figure(figsize=(12, 3))

        plt.subplot(1, 4, 1)
        plt.imshow(original_images[2*i])
        plt.title("Original Image")

        plt.subplot(1, 4, 2)
        plt.imshow(augmented_images[2*i])
        plt.title("Augmented Image")

        plt.subplot(1, 4, 3)
        plt.imshow(original_images[2*i+1])
        plt.title("Original Image")

        plt.subplot(1, 4, 4)
        plt.imshow(augmented_images[2*i + 1])
        plt.title("Augmented Image")

        plt.show()

# Zavolání funkce pro zobrazení 16 augmentovaných obrázků (pro jiny pocett stejne nebude fungovat)
show_augmented_images(train_ds)

In [ ]:
model.summary()

Testovano na nasledujici dvojice:
adam + SparseCategoricalCrossentropy

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

Samotny trenink nad train_datasetem (train_ds)
verbose=1 - idealni vypis behem treninku (0 - nic, 2- jen cisla)

In [ ]:
history = model.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose = 1,
    validation_data=val_ds
)

Over na testovaci sade presnost pro dany dataset

In [ ]:
scores = model.evaluate(test_ds)

Zobrazneni jednotlivzch konvolucnich vrstvev modelu

In [ ]:
#ZDROJ: https://www.kaggle.com/code/arpitjain007/guide-to-visualize-filters-and-feature-maps-in-cnn

for layer in model.layers:
    
    if 'conv' not in layer.name:
        continue    
    filters , bias = layer.get_weights()
    print(layer.name , filters.shape)

# retrieve weights from the second hidden layer
# first 2 isnt from 
filters , bias = model.layers[2].get_weights()

# normalize filter values to 0-1 so we can visualize them
f_min, f_max = filters.min(), filters.max()
filters = (filters - f_min) / (f_max - f_min)

n_filters =6
ix=1
fig = plt.figure(figsize=(20,15))
for i in range(n_filters):
    # get the filters
    f = filters[:,:,:,i]
    for j in range(3):
        # subplot for 6 filters and 3 channels
        plt.subplot(n_filters,3,ix)
        plt.imshow(f[:,:,j] ,cmap='gray')
        ix+=1
#plot the filters 
plt.show()

Zobrazeni Konvolucnich vrstev (manualne vyber)

In [ ]:
model.summary()

# Vyber zkoumanou vrstvu
selected_layer = model.get_layer('conv2d_1')

# Získání váh vrstvy
weights, biases = selected_layer.get_weights()

# Výpis váh vrstvy
print("Váhy vrstvy:", weights)

# Vytvoření nového modelu obsahujícího pouze vybranou vrstvu
visualization_model = models.Model(inputs=model.input, outputs=selected_layer.output)

# Zobrazit architekturu modelu
visualization_model.summary()

# Příprava vzorku dat pro vizualizaci (nahradit tímto přípravou vašich vlastních dat)
sample_data = tf.random.normal(shape=(1, 127, 127, 32))

# Získání výstupu vybrané vrstvy pro vzorek dat
activations = visualization_model.predict(sample_data)

# Výpis tvaru aktivačních map
print("Tvar aktivačních map:", activations.shape)

# Zobrazit prvních 16 aktivačních map
for i in range(16):
    plt.subplot(4, 4, i + 1)
    plt.imshow(activations[0, :, :, i], cmap='viridis')
    plt.axis('off')

plt.show()

Ziskani dat z historie treninku pro zobrazeni grafu

In [ ]:
history.history.keys()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

Zobrazeni grafu loss funkci a presnosti pro trenink i validacni data k jednotlivym epocham treninku

In [ ]:
#plt.figure(1)
figure, axis = plt.subplots(1,2)
axis[0].plot(range(EPOCHS), loss, label='Training loss')
axis[0].plot(range(EPOCHS), val_loss, label='Validation loss')
axis[0].legend(loc='lower right')
axis[0].set_title('Loss function')

#plt.figure(2)
#plt.subplot(1,2,1)
axis[1].plot(range(EPOCHS), acc, label='Training Accuracy')
axis[1].plot(range(EPOCHS), val_acc, label='Validation Acc')
axis[1].legend(loc='lower right')
axis[1].set_title('Accuracy')
#plt.show

Zobrazeni skutecneho a odhadovaneho labelu pro 1 testovaci sadu

In [ ]:
for images_batch, label_batch in test_ds.take(1):
    first_image = (images_batch[0].numpy().astype('uint8'))
    first_label = label_batch[0].numpy().astype('uint8')

    print("actual label:", class_names[first_label])
    print("predicted label:", model.predict(images_batch)[0])

Tohle nerozumim, proc funguje s tim img, pozjisuji a vztvorim svoje vlastni implementaci, zdroj: temna zakouti internetu - nepouzitelne

In [ ]:
def predict(model, img):    #proc je zde argument img a nize se pouzije images nechapem
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array, 0) #create a batch

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100*(np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=[15, 15])
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i].numpy().astype('uint8')]
        print(predicted_class)
        print(confidence)
        print(actual_class)
        plt.title(f"Actual_class: {actual_class}\n Predicted_class: {predicted_class}\n Confidence: {confidence}")
        plt.axis("off")

Ficurka pro automaticke ukladani modelu po spusteni teto bunky
TODO: jak to potom pouzit :D

In [ ]:
#autosave new version
import os
model_version = max([int(i) for i in os.listdir("../models") + [0]]) +1
model.save(f"../models/{model_version}")